In [4]:
%load_ext autoreload
%autoreload 2

import numpy as np
import open3d as o3d
import k3d

from utils.data_utils import *
from utils.visualization_utils import *
from ray_tracing import *

## Test ray_tracing.intersection_points [DEPRECATED]

In [2]:
# init the object and normalize
bun = o3d.data.BunnyMesh()
mesh_bun = o3d.io.read_triangle_mesh(bun.path)
mesh_bun.vertices = o3d.utility.Vector3dVector(normalize(np.asarray(mesh_bun.vertices)))

# init camera views
camera_sphere = o3d.geometry.TriangleMesh.create_sphere(radius=2, resolution=10)
views = np.asarray(camera_sphere.vertices)

In [3]:
# setup open3d virtual camera
width = 320
height = 240
vis, ctr, param = setup_virtual_camera(width, height)

[Open3D WARNING] GLFW Error: Cocoa: Failed to find service port for display


In [4]:
# calculcate camera fov in degrees
fx = param.intrinsic.intrinsic_matrix[0, 0]
x0 = param.intrinsic.intrinsic_matrix[0, 2]
fov = np.arctan2(x0, fx) * 180 / np.pi

In [14]:
# setup raycasting scene and calculate the rays
scene = o3d.t.geometry.RaycastingScene()
center = np.zeros(3)
up = np.array([0, 1, 0])
eye = views[7]
rays = scene.create_rays_pinhole(fov, center, eye, up, width, height)
rays = rays.numpy().reshape(-1, 6)
rays = Ray(origin=Vec3D(rays[:, :3]), direction=Vec3D(rays[:, 3:]))

In [6]:
# setup the grid
num_voxels = 32
voxel_size = 2. / num_voxels
min_bound = Vec3D(np.array([-1., -1., -1.]))
max_bound = Vec3D(-min_bound.data)
grid = Grid(voxel_size, num_voxels, min_bound, max_bound)

In [7]:
# find and process t_min and t_max
t_min, t_max = intersection_points(rays, grid)
# mask out the non-intersecting rays
min_mask = np.logical_or(t_min == np.inf, t_min == -np.inf)
max_mask = np.logical_or(t_max == np.inf, t_max == -np.inf)
mask = np.logical_not(np.logical_or(min_mask, max_mask))
# clip rays within a threshold
thresh = 4  # twice the radius
t_min = np.clip(t_min, a_min=0, a_max=thresh)
t_max = np.clip(t_max, a_min=0, a_max=thresh)

In [8]:
# plot the object
vertices = flip_axes(np.asarray(mesh_bun.vertices))
indices = np.asarray(mesh_bun.triangles)
plot = k3d.plot()
plot += k3d.mesh(
            vertices, 
            indices,
            attribute=vertices[:, 2],
            color_map=k3d.colormaps.matplotlib_color_maps.inferno,
        )

# plot entrance points
points = flip_axes(rays.origin.data + t_min[:, None] * rays.direction.data)[mask]
plot += k3d.points(points, point_size=0.01, shader='mesh', color=0xff0ac5)

# plot exit points
points = flip_axes(rays.origin.data + t_max[:, None] * rays.direction.data)[mask]
plot += k3d.points(points, point_size=0.01, shader='mesh', color=0x000ac5)

# plot camera
plot += k3d.points(flip_axes(eye), point_size=0.1, shader='mesh', color=0x00ffc5)

plot.display()

/Users/khasmamad/miniconda3/envs/nbv/lib/python3.8/site-packages/traittypes/traittypes.py:97: UserWarning: Given trait value dtype "float64" does not match required type "float32". A coerced copy has been created.
  warnings.warn(
/Users/khasmamad/miniconda3/envs/nbv/lib/python3.8/site-packages/traittypes/traittypes.py:97: UserWarning: Given trait value dtype "int32" does not match required type "uint32". A coerced copy has been created.
  warnings.warn(


Output()

## Test ray_tracing.shoot_rays [DEPRECATED]

In [27]:
# setup the grid
num_voxels = 32
voxel_size = 2. / num_voxels
min_bound = Vec3D(np.array([-1., -1., -1.]))
max_bound = Vec3D(-min_bound.data)
grid = Grid(voxel_size, num_voxels, min_bound, max_bound)

# setup ground truth grid
voxel_bun = o3d.geometry.VoxelGrid.create_from_triangle_mesh_within_bounds(
    mesh_bun, voxel_size, min_bound.data, max_bound.data)
occupied_indices = np.asarray([voxel.grid_index for voxel in voxel_bun.get_voxels()])
gt_grid = Grid(voxel_size, num_voxels, min_bound, max_bound)
gt_grid.data[gt_grid.data == VoxelType.unseen] = 0
gt_grid.set_labels(occupied_indices, VoxelType.occupied)

In [ ]:
# setup open3d virtual camera
width = 320
height = 240
vis, ctr, param = setup_virtual_camera(width, height)

In [104]:
# calculcate camera fov in degrees
fx = param.intrinsic.intrinsic_matrix[0, 0]
x0 = param.intrinsic.intrinsic_matrix[0, 2]
fov = np.arctan2(x0, fx) * 180 / np.pi

# setup raycasting scene and calculate the rays
scene = o3d.t.geometry.RaycastingScene()
center = np.zeros(3)
up = np.array([0, 1, 0])
eye = np.array([0, 0, -1]) * np.sqrt(5)
rays = scene.create_rays_pinhole(fov, center, eye, up, width, height)
rays = rays.numpy().reshape(-1, 6)
num_rays = len(rays)
rays = Ray(origin=Vec3D(rays[:num_rays, :3]), direction=Vec3D(rays[:num_rays, 3:]))

In [105]:
print(rays.origin.data)
print(rays.direction.data)

[[-0.       -0.       -2.236068]
 [-0.       -0.       -2.236068]
 [-0.       -0.       -2.236068]
 ...
 [-0.       -0.       -2.236068]
 [-0.       -0.       -2.236068]
 [-0.       -0.       -2.236068]]
[[-0.31169513 -0.23352703  0.92103815]
 [-0.30992916 -0.23366897  0.9215979 ]
 [-0.30816004 -0.23381028  0.9221552 ]
 ...
 [ 0.30815998  0.2338103   0.9221552 ]
 [ 0.30992913  0.233669    0.9215979 ]
 [ 0.3116951   0.23352706  0.92103815]]


In [106]:
shoot_rays(rays, grid, gt_grid, 5)

start_idx:
[ 9  9  9 ... 22 22 22]
end_idx:
[ 0  0  0 ... 31 31 31]
step:
[-1 -1 -1 ...  1  1  1]
tmax:
[1.40361512 1.41161288 1.41971691 ... 1.41971712 1.41161298 1.40361532]
tdelta:
[0.20051645 0.20165898 0.2028167  ... 0.20281674 0.20165901 0.20051646]
start_idx:
[10 10 10 ... 21 21 21]
end_idx:
[ 4  3  3 ... 28 28 27]
step:
[-1 -1 -1 ...  1  1  1]
tmax:
[1.60580976 1.60483439 1.60386444 ... 1.60386427 1.60483422 1.6058096 ]
tdelta:
[0.26763496 0.26747242 0.26731074 ... 0.2673107  0.26747236 0.26763493]
start_idx:
[0 0 0 ... 0 0 0]
end_idx:
[27 27 28 ... 28 27 27]
step:
[1 1 1 ... 1 1 1]
tmax:
[1.40989601 1.40903964 1.40818815 ... 1.40818815 1.40903964 1.40989601]
tdelta:
[0.06785821 0.067817   0.06777601 ... 0.06777601 0.067817   0.06785821]
67012
x: [1.40361512 1.41161288 1.41971691 ... 1.41971712 1.41161298 1.40361532]
y: [1.60580976 1.60483439 1.60386444 ... 1.60386427 1.60483422 1.6058096 ]
z: [1.40989601 1.40903964 1.40818815 ... 1.40818815 1.40903964 1.40989601]

67012
x: [1.

In [11]:
occupied_x, occupied_y, occupied_z = np.where(grid.data == VoxelType.occupied)
empty_x, empty_y, empty_z = np.where(grid.data == VoxelType.empty)
unseen_x, unseen_y, unseen_z = np.where(grid.data == VoxelType.unseen)

occ_grid = np.zeros([num_voxels, num_voxels, num_voxels])
occ_grid[np.where(grid.data == VoxelType.occupied)] = 1
empty_grid = np.zeros([num_voxels, num_voxels, num_voxels])
empty_grid[np.where(grid.data == VoxelType.empty)] = 1
unseen_grid = np.zeros([num_voxels, num_voxels, num_voxels])
unseen_grid[np.where(grid.data == VoxelType.unseen)] = 1

In [12]:
visualize_occupancy(empty_grid, do_flip_axes=True)
visualize_occupancy(occ_grid, do_flip_axes=True)
visualize_occupancy(unseen_grid, do_flip_axes=True)

Output()

Output()

Output()

## Test data generation pipeline

### Test view movement

In [20]:
from data_generation import move

In [26]:
sphere = o3d.geometry.TriangleMesh.create_sphere(radius=1.0, resolution=10, create_uv_map=False)
sphere_points = np.array(sphere.vertices)

view = random_sphere_point(npoints=1, radius=np.sqrt(3.1))[0]
path = [view]
new_view = view
for _ in range(4):
    new_view = move("left", new_view)
    path.append(new_view)

visualize_camera_path(path)

Output()

### test data_generation.generate_data

In [5]:
from data_generation import generate_data
import json

In [6]:
bun = o3d.data.BunnyMesh()
mesh_bun = o3d.io.read_point_cloud(bun.path)
mesh_bun.points = o3d.utility.Vector3dVector(normalize(np.asarray(mesh_bun.points)))

In [7]:
# setup ground truth grid
num_voxels = 32
grid_data, voxel_size, min_bound, max_bound = to_occupancy_grid(
    mesh_bun, input_type="point_cloud", do_normalization=True, num_voxels=num_voxels
)
gt_grid = Grid(voxel_size, num_voxels, Vec3D(min_bound), Vec3D(max_bound))
gt_grid.data = grid_data

In [27]:
generate_data(mesh_bun, gt_grid, debug=True)

Received scan no 0
	next_view_dir : right
	next_view_coords : [-0.9518258346554734, 0.1423862566091713, 1.4743655362260872]
	# of occed : 244
	next_view_utility : 97

Received scan no 1
	next_view_dir : right
	next_view_coords : [-0.26823556664683373, -0.9242813134400227, 1.4743655362260872]
	# of occed : 341
	next_view_utility : 73

Received scan no 2
	next_view_dir : down
	next_view_coords : [-0.4802361881758354, -1.654789259744241, 0.36200236104402694]
	# of occed : 414
	next_view_utility : 56

Received scan no 3
	next_view_dir : down
	next_view_coords : [-0.4109209638137904, -1.4159441005596172, -0.9624168876265425]
	# of occed : 470
	next_view_utility : 145

Received scan no 4
	next_view_dir : right
	next_view_coords : [0.8986710587489161, -1.1688216555909374, -0.9624168876265425]
	# of occed : 615
	next_view_utility : 73

Coverage is 0.7510917030567685, writing data to data/data/trial/bun.json


In [8]:
with open('data/data/trial/bun.json', 'r') as f:
    bun_scans = json.load(f)

In [9]:
camera_path = list()
for i, scan in bun_scans.items():
    direction = scan['next_view_dir']
    view = np.array(scan['next_view_coords'])
    partial_model = np.array(scan['partial_model'])
    utility = scan['next_view_utility']
    camera_path.append(view)
    visualize_partial_model(partial_model, True)

/Users/khasmamad/miniconda3/envs/nbv/lib/python3.8/site-packages/traittypes/traittypes.py:97: UserWarning: Given trait value dtype "int64" does not match required type "float32". A coerced copy has been created.
  warnings.warn(


Output()

Output()

Output()

Output()

Output()

In [30]:
visualize_camera_path(camera_path)

/Users/khasmamad/miniconda3/envs/nbv/lib/python3.8/site-packages/traittypes/traittypes.py:97: UserWarning: Given trait value dtype "float64" does not match required type "float32". A coerced copy has been created.
  warnings.warn(
/Users/khasmamad/miniconda3/envs/nbv/lib/python3.8/site-packages/traittypes/traittypes.py:97: UserWarning: Given trait value dtype "int32" does not match required type "float32". A coerced copy has been created.
  warnings.warn(


Output()

## ScanNet

In [4]:
import json
from pathlib import Path

from data_generation import generate_data

In [5]:
root_dir = Path('data/data/ShapeNetCore.v2')
chairs_dir = root_dir / '03001627'
chair_ids = list(chairs_dir.glob('*'))
chair_objs = [chair_id /'models'/ 'model_normalized.obj' for chair_id in chair_ids]

In [6]:
chair_objs[0]

PosixPath('data/data/ShapeNetCore.v2/03001627/a97a00bee5585e49f05defab1869a09/models/model_normalized.obj')

In [ ]:
generate_data(chair_objs[:1000], coverage_thresh=0.75, debug=True)

  0%|                                                                                                                                                                                      | 0/1000 [00:00<?, ?it/s]

Processing 03001627 / a97a00bee5585e49f05defab1869a09
Finished setting the Ground Truth Grid
Starting scan session # 0
	Received scan no 0
		next_view_dir : down
		next_view_coords : [-0.9312254229291504, 1.4181064416125269, 0.47094939425322746]
		# of occed : 220
		next_view_utility : 190

	Received scan no 1
		next_view_dir : down
		next_view_coords : [-0.8412660842660971, 1.2811128474728781, -0.8666148207269029]
		# of occed : 410
		next_view_utility : 91

	Received scan no 2
		next_view_dir : right
		next_view_coords : [-1.5267759558936498, 0.13391763514007063, -0.8666148207269028]
		# of occed : 501
		next_view_utility : 98

	Received scan no 3
		next_view_dir : right
		next_view_coords : [-1.051444208637272, -1.115098125108108, -0.8666148207269028]
		# of occed : 599
		next_view_utility : 96

	Received scan no 4
		next_view_dir : up
		next_view_coords : [-1.163878582754021, -1.2343392210648612, 0.47094939425322735]
		# of occed : 695
		next_view_utility : 44

	Received scan no 5


  0%|▏                                                                                                                                                                           | 1/1000 [00:28<7:48:26, 28.13s/it]

	Received scan no 6
		next_view_dir : right
		next_view_coords : [0.6451159535830334, -0.5786593591537039, 1.5326378412712107]
		# of occed : 761
		next_view_utility : 9

	Coverage is 0.9080188679245284, writing data to data/data/ShapeNetCore.v2_nbv/03001627/a97a00bee5585e49f05defab1869a09/0
Processing 03001627 / 662928421872897f489452a0032ea575
Finished setting the Ground Truth Grid
Starting scan session # 0
	Received scan no 0
		next_view_dir : up
		next_view_coords : [-1.1152148825596948, 0.15657518429446046, 1.3534326645166068]
		# of occed : 174
		next_view_utility : 151

	Received scan no 1
		next_view_dir : right
		next_view_coords : [-0.5223840498431318, -0.9976647368174174, 1.3534326645166068]
		# of occed : 325
		next_view_utility : 112

	Received scan no 2
		next_view_dir : right
		next_view_coords : [0.7639759119611216, -0.8273819121558387, 1.3534326645166068]
		# of occed : 437
		next_view_utility : 143

	Received scan no 3
		next_view_dir : down
		next_view_coords : [1.18

/Users/khasmamad/Desktop/nbv/next-best-view/ray_tracing/ray_casting.py:45: RuntimeWarning: divide by zero encountered in true_divide
  inv_dir = 1. / dir  # division by 0 results in inf
/Users/khasmamad/Desktop/nbv/next-best-view/ray_tracing/ray_casting.py:97: RuntimeWarning: divide by zero encountered in true_divide
  t_delta = np.abs(partial_grid.voxel_size / direction)  # divison by 0 = inf: good
/Users/khasmamad/Desktop/nbv/next-best-view/ray_tracing/ray_casting.py:99: RuntimeWarning: divide by zero encountered in true_divide
  t_max_axis = t_min + (


...

  8%|█████████████▎                                                                                                                                                             | 78/1000 [31:25<3:34:47, 13.98s/it]

	Received scan no 3
		next_view_dir : right
		next_view_coords : [-0.8871760797718462, -1.4882269669033872, -0.3132077560696779]
		# of occed : 660
		next_view_utility : 13

	Coverage is 0.9168937329700273, writing data to data/data/ShapeNetCore.v2_nbv/03001627/a7d75c84027af5e7321f25048c6142/0
Processing 03001627 / 117bd6da01905949a81116f5456ee312
Finished setting the Ground Truth Grid
Starting scan session # 0
	Received scan no 0
		next_view_dir : left
		next_view_coords : [-1.476955169284223, 0.874085534333233, 0.39316396909431917]
		# of occed : 230
		next_view_utility : 112

	Received scan no 1
		next_view_dir : down
		next_view_coords : [-1.2836151862482417, 0.7596638606802105, -0.9355441584496852]
		# of occed : 342
		next_view_utility : 83



  8%|█████████████▌                                                                                                                                                             | 79/1000 [31:33<3:07:23, 12.21s/it]

	Received scan no 2
		next_view_dir : right
		next_view_coords : [-1.3780081964739774, -0.5708331963377802, -0.9355441584496852]
		# of occed : 425
		next_view_utility : 40

	Coverage is 0.908203125, writing data to data/data/ShapeNetCore.v2_nbv/03001627/117bd6da01905949a81116f5456ee312/0
Processing 03001627 / 1f83e49ecdf209ddd7f81f50826c13f0
[Open3D INFO] Skipping non-triangle primitive geometry of type: 2
[Open3D INFO] Skipping non-triangle primitive geometry of type: 2
[Open3D INFO] Skipping non-triangle primitive geometry of type: 2
[Open3D INFO] Skipping non-triangle primitive geometry of type: 2
[Open3D INFO] Skipping non-triangle primitive geometry of type: 2
[Open3D INFO] Skipping non-triangle primitive geometry of type: 2
[Open3D INFO] Skipping non-triangle primitive geometry of type: 2
[Open3D INFO] Skipping non-triangle primitive geometry of type: 2
[Open3D INFO] Skipping non-triangle primitive geometry of type: 2
[Open3D INFO] Skipping non-triangle primitive geometry of typ

  8%|█████████████▋                                                                                                                                                             | 80/1000 [31:50<3:26:17, 13.45s/it]

In [1]:
generate_data(chair_objs[80:1000], coverage_thresh=0.90, debug=True)

NameError: name 'generate_data' is not defined

In [9]:
scans = list()
file_dir_path = Path('data/data/ShapeNetCore.v2_nbv/03001627/97f1121561e481a3a1f1563b05df75c6/0')
for json_file_path in sorted(file_dir_path.glob('*')):
    with open(json_file_path, 'r') as f:
        scans.append(json.load(f))

In [10]:
camera_path = list()
for scan in scans:
    direction = scan['next_view_dir']
    view = np.array(scan['next_view_coords'])
    partial_model = np.array(scan['partial_model'])
    utility = scan['next_view_utility']
    camera_path.append(view)
    visualize_partial_model(partial_model, True)
    
visualize_camera_path(camera_path)

Output()

Output()

Output()

Output()

Output()

Output()

Output()

Output()

Output()

In [13]:
from utils.binwox_rw import *

In [15]:
with open('data/data/ShapeNetCore.v2/02691156/1a04e3eab45ca15dd86060f189eb133/models/model_normalized.surface.binvox', 'rb') as f:
    m1 = read_as_3d_array(f)

In [25]:
m1.dims

[128, 128, 128]

In [24]:
x, y, z = sparse_to_dense(m1.data, dims=32)

ValueError: voxel_data is wrong shape; should be 3xN array.

array([  0,   0,   0, ..., 127, 127, 127])